In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OneHotEncoder

import random

from sklearn.metrics import classification_report, roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns=None
pd.options.mode.chained_assignment = None

In [ ]:
triggers_actions = pd.read_csv(r'/home/kusapochka/triggers/before_logreg.csv')

In [ ]:
def encode_columns(df, columns):
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    encoded_data = encoder.fit_transform(df[columns])
    
    # Создаем DataFrame с новыми названиями столбцов
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns))
    encoded_df.index = df.index  # сохраняем индекс для правильного объединения
    
    # Объединяем закодированные данные с исходным DataFrame и удаляем оригинальные столбцы
    df = pd.concat([df.drop(columns, axis=1), encoded_df], axis=1)
    return df

In [ ]:

# Пример использования:
triggers_actions = encode_columns(triggers_actions, ['type', 'last_successful_type'])

In [ ]:
triggers_actions['trigger'] = triggers_actions['trigger'].astype('category')

In [ ]:
triggers_actions.info()

In [ ]:
X = triggers_actions.drop(columns = ['guid','date','result','action_date'])
y_succes = triggers_actions['result']  # таргет для взаимодействия

# Разделение данных на train/test с фиксацией индексов
X_train, X_test, y_train_success, y_test_success = train_test_split(X, y_succes, test_size=0.3, random_state=42)

In [ ]:
# Сохраняем индексы, которые попали в train/test
train_indices = X_train.index
test_indices = X_test.index

### logreg

In [ ]:
### модели 
# модель для взаимодействия 
logreg_success = LogisticRegression(class_weight='balanced',multi_class='ovr')

logreg_success.fit(X_train, y_train_success)

In [ ]:

# получаем вероятности для каждого из классов
probs = logreg_success.predict_proba(X_test)

In [ ]:

# добавляем вероятности для каждого класса в датафрейм
triggers_actions.loc[test_indices,'prob_no_interaction'] = probs[:, 0]  # вероятность для класса 0 (нет взаимодействия)
triggers_actions.loc[test_indices,'prob_success'] = probs[:, 1]         # вероятность для класса 1 (успех)
triggers_actions.loc[test_indices,'prob_unsuccessful'] = probs[:, 2]    # вероятность для класса -1 (неуспех)


In [ ]:
triggers_actions.head(10)

In [ ]:
# получаем коэффициенты модели
feature_importance = logreg_success.coef_[0]

# создаем DataFrame для наглядности
feature_names = X_train.columns  # названия признаков из обучающего набора
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

# сортируем признаки по важности
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# визуализация
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Feature Importance in Logistic Regression')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
after_logreg_success = triggers_actions.dropna()[triggers_actions['result'] == 1]
after_logreg_fail = triggers_actions.dropna()[triggers_actions['result'] == -1]

In [ ]:
after_logreg_success.head()

In [ ]:
# создаем фигуру и две оси рядом
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# первый график - для успешных взаимодействий
sns.histplot(after_logreg_success['prob_success'], bins=50, kde=True, color='blue', ax=axes[0])
axes[0].set_title('Распределение вероятностей успешных взаимодействий', fontsize=14)
axes[0].set_xlabel('Вероятность взаимодействия', fontsize=12)
axes[0].set_ylabel('Количество', fontsize=12)

# второй график - для неуспешных взаимодействий
sns.histplot(after_logreg_fail['prob_success'], bins=50, kde=True, color='blue', ax=axes[1])
axes[1].set_title('Распределение вероятностей неуспешных взаимодействий', fontsize=14)
axes[1].set_xlabel('Вероятность взаимодействия', fontsize=12)
axes[1].set_ylabel('Количество', fontsize=12)

# отображаем оба графика
plt.tight_layout()
plt.show()

### пробуем деревья

In [ ]:
# создаем и обучаем модель
tree_clf = DecisionTreeClassifier(random_state=42, class_weight='balanced')
tree_clf.fit(X_train, y_train_success)


In [ ]:
# извлечение важности признаков
feature_importances = tree_clf.feature_importances_

# создаем DataFrame для удобства отображения
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# сортируем признаки по важности
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# визуализируем важность признаков
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance in Decision Tree')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# получаем вероятности для каждого класса на тестовом наборе
y_proba = tree_clf.predict_proba(X_test)


In [ ]:

# создаем DataFrame с вероятностями, используя тестовые индексы для соответствия с triggers_actions
proba_df = pd.DataFrame(y_proba, columns=tree_clf.classes_, index=test_indices)


In [ ]:

# объединяем вероятности с исходным DataFrame triggers_actions
# здесь будет произведено слияние по индексу
triggers_actions_pred = triggers_actions.join(proba_df, how='inner')